In [1]:
import searchconsole
import json
import pydata_google_auth
from pydata_google_auth import cache
from googleapiclient.discovery import build
from searchconsole.account import Account
import pandas as pd

with open('credentials.json') as f:
    cs = json.load(f)
    
credentials = pydata_google_auth.get_user_credentials(scopes=['https://www.googleapis.com/auth/webmasters.readonly'],
                                                     client_id=cs['installed']['client_id'],
                                                     client_secret=cs['installed']['client_secret'],
                                                     credentials_cache=cache.REAUTH,
                                                     auth_local_webserver=False)

service = build('webmasters', 'v3', credentials=credentials)

account = Account(service, credentials)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=867904848807-m93rvv96u69s4kei8tg63cltjb4rhiul.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fwebmasters.readonly&state=dhM3fc6ztm2khlwXb3QXYoPDT0xALu&prompt=consent&access_type=offline
Enter the authorization code: 4/1AX4XfWi6HCZBkb3zL70JX0rUBmu0A396RCmCYgBUmnI1qlhFcMocj5jSrl4


### Список сайтов в консоли:

In [2]:
account.webproperties

[<searchconsole.account.WebProperty(url='http://www.integra.od.ua/')>,
 <searchconsole.account.WebProperty(url='https://nash.od.ua/')>,
 <searchconsole.account.WebProperty(url='sc-domain:zolotoybereg.com')>,
 <searchconsole.account.WebProperty(url='http://tablichka.com.ua/')>,
 <searchconsole.account.WebProperty(url='https://led-one.com.ua/')>,
 <searchconsole.account.WebProperty(url='https://kompmaister.com/')>,
 <searchconsole.account.WebProperty(url='https://img5.lalafo.com/')>,
 <searchconsole.account.WebProperty(url='http://viz.od.ua/')>,
 <searchconsole.account.WebProperty(url='http://ggwp.com.ua/')>,
 <searchconsole.account.WebProperty(url='https://exomania.com.ua/')>,
 <searchconsole.account.WebProperty(url='https://lasode.com/')>,
 <searchconsole.account.WebProperty(url='http://arendalulek.kiev.ua/')>,
 <searchconsole.account.WebProperty(url='https://beetroot.academy/')>,
 <searchconsole.account.WebProperty(url='sc-domain:legio.info')>,
 <searchconsole.account.WebProperty(url=

### Выбор сайта:

In [3]:
webproperty = account['https://cook.me/']

In [4]:
# собираем данные
query = webproperty.query.range(start='2021-06-01', stop='2021-06-30').dimension('page', 'query')
data = query.get()
len(data)

51297

In [5]:
#создаем data frame с данными
df = data.to_dataframe()
df.head()
#df.to_excel('raw_data.xlsx') #если надо сохранить изначальные данные в файл, убираем знак "#"

,page,query,clicks,impressions,ctr,position
0,https://cook.me/recipe/best-fried-walleye/,deep fried walleye,76,392,0.193878,1.183673
1,https://cook.me/recipe/black-lemonade/,black lemonade,65,1991,0.032647,2.008036
2,https://cook.me/recipe/cowboy-rice-salad/,cowboy rice salad,35,436,0.080275,2.864679
3,https://cook.me/recipe/pineapple-beef-lettuce-...,korean pineapple beef lettuce wraps,31,96,0.322917,2.270833
4,https://cook.me/recipe/halupki-stuffed-cabbage...,halupki,30,2467,0.012161,9.796514


In [6]:
#Чистим от ненужных запросов при помощи регулярного выражения
patternDel = ".*cookme.*"
filter = df['query'].str.contains(patternDel)
df = df[~filter]
df.head()

,page,query,clicks,impressions,ctr,position
0,https://cook.me/recipe/best-fried-walleye/,deep fried walleye,76,392,0.193878,1.183673
1,https://cook.me/recipe/black-lemonade/,black lemonade,65,1991,0.032647,2.008036
2,https://cook.me/recipe/cowboy-rice-salad/,cowboy rice salad,35,436,0.080275,2.864679
3,https://cook.me/recipe/pineapple-beef-lettuce-...,korean pineapple beef lettuce wraps,31,96,0.322917,2.270833
4,https://cook.me/recipe/halupki-stuffed-cabbage...,halupki,30,2467,0.012161,9.796514


In [7]:
#Считаем кол-во страниц, запросов, кликов, показов, средний CTR и среднюю позицию
data = {
    'Total pages': [int(df['page'].nunique())],
    'Total queries': [int(df['query'].nunique())],
    'Total clicks': [int(df['clicks'].sum())],
    'Total impressions': [int(df['impressions'].sum())],
    'Average CTR': [round(df['ctr'].mean(),2)],
    'Average position': [round(df['position'].mean(),2)],
    'Average queries per page': [round(int(df['query'].nunique()) / int(df['page'].nunique()),2)],
}

df_stats = pd.DataFrame.from_dict(data)
df_stats.T.head(10)

,0
Total pages,2725.00
Total queries,49291.00
Total clicks,3671.00
Total impressions,461397.00
Average CTR,0.00
Average position,52.55
Average queries per page,18.09


In [8]:
#считаем сумму уникальных страниц по каждому ключу, также считаем сумму кликов, показов, средний CTR и позицию
df_summary = df.groupby('query').agg(
    unique_pages=('page', 'nunique'),
    total_clicks=('clicks', 'sum'),
    total_impressions=('impressions', 'sum'),
    avg_ctr=('ctr', 'mean'),
    avg_position=('position', 'mean'),    
).sort_values(by='total_clicks', ascending=False)

df_summary.head()

,unique_pages,total_clicks,total_impressions,avg_ctr,avg_position
query,,,,,
deep fried walleye,1,76,392,0.193878,1.183673
black lemonade,1,65,1991,0.032647,2.008036
cowboy rice salad,1,35,436,0.080275,2.864679
korean pineapple beef lettuce wraps,1,31,96,0.322917,2.270833
halupki,1,30,2467,0.012161,9.796514


In [9]:
#получаем запросы где канибализация
df_cannibalised = df.groupby('query').agg(
    page = ('page', 'unique'),
    unique_pages=('page', 'nunique'),
    total_clicks=('clicks', 'sum'),
    total_impressions=('impressions', 'sum'),
    avg_ctr=('ctr', 'mean'),
    avg_position=('position', 'mean'),    
).sort_values(by='unique_pages', ascending=False)

df_cannibalised = df_cannibalised[df_cannibalised['unique_pages'] > 1] #берем страницы где значение unique_pages больше 1
df_cannibalised = df_cannibalised[df_cannibalised['unique_pages'] < 10] #берем страницы где значение не больше 10 (чтобы не смотреть все страницы, можно эту строку закомментить)
df_cannibalised.head()

,page,unique_pages,total_clicks,total_impressions,avg_ctr,avg_position
query,,,,,,
gooseberry recipes,[https://cook.me/dessert/berries/gooseberry-re...,9,1,231,0.002584,33.877024
cook something for me,"[https://cook.me/, https://cook.me/all-recipes...",8,0,40,0.000000,9.848485
show me,"[https://cook.me/, https://cook.me/all-recipes...",8,0,9,0.000000,6.125000
cooke me,"[https://cook.me/, https://cook.me/cuisine/ukr...",8,0,16,0.000000,1.319444
cooking for me,"[https://cook.me/, https://cook.me/all-recipes...",8,0,80,0.000000,6.159247


In [17]:
#Распаковываем урлы
final = df_cannibalised.explode('page')
final.drop(['avg_ctr', 'avg_position', 'total_impressions', 'total_clicks', 'unique_pages'], axis=1, inplace=True)
final.head()

,page
query,
gooseberry recipes,https://cook.me/dessert/berries/gooseberry-rec...
gooseberry recipes,https://cook.me/recipe/easy-gooseberry-apple-c...
gooseberry recipes,https://cook.me/recipe/gooseberry-coconut-cake/
gooseberry recipes,https://cook.me/recipe/gooseberry-fool/
gooseberry recipes,https://cook.me/recipe/gooseberry-jam/


In [12]:
#Соединяем значения в таблице по канибализации со значениями из изначальной таблицы
data_zapis = pd.merge(final, df, on=['query', 'page'],how = "left")
data_zapis.head()

,query,page,clicks,impressions,ctr,position
0,gooseberry recipes,https://cook.me/dessert/berries/gooseberry-rec...,1,43,0.023256,72.395349
1,gooseberry recipes,https://cook.me/recipe/easy-gooseberry-apple-c...,0,7,0.000000,30.428571
2,gooseberry recipes,https://cook.me/recipe/gooseberry-coconut-cake/,0,25,0.000000,14.520000
3,gooseberry recipes,https://cook.me/recipe/gooseberry-fool/,0,142,0.000000,43.049296
4,gooseberry recipes,https://cook.me/recipe/gooseberry-jam/,0,8,0.000000,53.500000


In [14]:
new_data = data_zapis.set_index(['query', 'page', 'clicks', 'impressions', 'ctr', 'position'])
new_data.head()

Empty DataFrame
Columns: []
Index: [(gooseberry recipes, https://cook.me/dessert/berries/gooseberry-recipes/, 1, 43, 0.023255813953488372, 72.3953488372093), (gooseberry recipes, https://cook.me/recipe/easy-gooseberry-apple-crumble/, 0, 7, 0.0, 30.428571428571427), (gooseberry recipes, https://cook.me/recipe/gooseberry-coconut-cake/, 0, 25, 0.0, 14.52), (gooseberry recipes, https://cook.me/recipe/gooseberry-fool/, 0, 142, 0.0, 43.04929577464789), (gooseberry recipes, https://cook.me/recipe/gooseberry-jam/, 0, 8, 0.0, 53.5)]

In [16]:
#записываем результат
new_data.to_excel('cannibalization.xlsx')